In [ ]:
import pandas as pd

# The dataframe column names were hard coded due to it being substantially easier to extract the customer details by hand, and it only needing to be done once
column_names = ["Date of Sale", "Product", "Product Category", "Quantity", "Total Price", "Total Profit", "Payment Type", "Customer Age Category", "Customer Location", "Customer ID", "Customer Gender"]

df = pd.DataFrame(columns= column_names) 

The function below sorts the customer data to make sure no data is lost and to make it so it is always in the same order.

In [ ]:
def loopy_loop(index, split_details):
    titles = ["Age", "Category:", "Location:", "ID:", "Gender:"]
    keep_going = True
    final_string = ""
    prev = False

    while keep_going:
        if index == len(split_details) or split_details[index] in titles:
            return final_string, index
        else:
            if prev:
                final_string = final_string + " " + split_details[index]
                index += 1
            else:
                final_string = final_string + split_details[index]
                index += 1
                prev = True

The function below extract the customer data and return the cleaned version of it, so it can be added to the dataframe without any issue or future issue. 

In [ ]:
def extract_customer_details(details):
    titles = ["Age", "Category:", "Location:", "ID:", "Gender:"]
    split_details = details.split(" ")
    list_data = [None, None, None, None]
    current_index = 0

    for i in range(0, len(titles) - 1):
        if split_details[current_index] == "Age":
            current_index += 2
            final_string, current_index = loopy_loop(current_index, split_details)
            list_data[0] = final_string
        elif split_details[current_index] == "Location:":
            current_index += 1
            final_string, current_index = loopy_loop(current_index, split_details)
            list_data[1] = final_string
        elif split_details[current_index] == "ID:":
            current_index += 1
            final_string, current_index = loopy_loop(current_index, split_details)
            list_data[2] = final_string
        elif split_details[current_index] == "Gender:":
            current_index += 1
            final_string, current_index = loopy_loop(current_index, split_details)
            list_data[3] = final_string
            
    return list_data

The code below is the core of the scraping. It simply pulls the data from the retails sales and then proceeds to take special action regarding the customer details by calling the function above.

In [ ]:
for k in range(1, 5):
    for i in range(1, 18): # the pages never go further than 17 pages and there are only 15 pages for Quarter 1, 2, and 4
        if k in [1,2,4] and i < 16 or k == 3:
            page = ""
            if i < 10:
                page = "0"+str(i)
            else:
                page = str(i)
            quarter = str(k)
            URL = 'http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q' + quarter + '-page' +page+ '.html' # Here we have two variable to make sure every quarter and page is scraped
            tables = pd.read_html(URL)
            
            for j in range(0, len(tables)):
                data = []
                for i in range(0, 7):
                    data.append(tables[j][1][i])
                
                temp_data = extract_customer_details(tables[j][1][7])
                data = [data + temp_data]
        
                temp_df = pd.DataFrame(data, columns=column_names)
                df = pd.concat([df, temp_df], ignore_index=True)

df.to_csv('scraped_retail_data.csv')